In [3]:
import pandas as pd
import numpy as np
from vit import *

""" 
Extracts the features of the images in the MNIST dataset using the trained vision transformer.
This way the database of MNIST images becomes a database of MNIST trasnformer feature representations.
Python notebook is used to test the code before running it fully as a pure Python script. 

Steps:
    1. Load the trained transformer model
    2. Strip off the final classification layer
    3. Go through images and forward propagate them
    4. For each image save the final transformer layer representation of that image.
"""

run_on_server = True 

model_file = "vit_model_0404.pt"

model_folder =  "/usr/itetnas04/data-scratch-01/ddordevic/data/cluster_scripts/vit_copy/model_save/" \
                if run_on_server else \
                "C:/Users/danil/Desktop/Master thesis/Code/msc-thesis/model_save/" 

model_path = model_folder + model_file

In [10]:
# Loading data
transform = ToTensor()

train_set = MNIST(root='./../datasets', train=True, download=True, transform=transform)
test_set = MNIST(root='./../datasets', train=False, download=True, transform=transform)

train_loader = DataLoader(train_set, shuffle=True, batch_size=1)
test_loader = DataLoader(test_set, shuffle=False, batch_size=1)

# Defining model and training options
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: ", device, f"({torch.cuda.get_device_name(device)})" if torch.cuda.is_available() else "")

# Loading the model: Replace the final layer with identity -> Keeps the features
model = torch.load(model_path, map_location=torch.device('cpu'))
model.mlp = nn.Identity()
model.eval()

# Pandas dataframe containing flattened images, their corresponding features and labels
training_df = pd.DataFrame(columns=['image','feature','label'])
test_df = pd.DataFrame(columns=['image','feature','label'])

with torch.no_grad():
    # Feature extraction loop: Training set
    i = 0
    for batch in tqdm(train_loader, leave=False):
        x, y = batch
        x, y = x.to(device), y.to(device)
        x_features = model(x)
        # new_row = {'image': x.flatten().cpu(), 'feature': x_features.cpu(), 'label': y.cpu()}
        new_row = {'image': x.cpu(), 'feature': x_features.cpu(), 'label': y.cpu()}
        # new_row = {'image': 1, 'feature': 2, 'label': 3}
        training_df = training_df.append(new_row, ignore_index=True)
        i+=1
        if i == 5:
            break

    # Feature extraction loop: Test set
    i = 0
    for batch in tqdm(test_loader, leave=False):
        x, y = batch
        x, y = x.to(device), y.to(device)
        x_features = model(x)
        new_row = {'image': x.flatten().cpu(), 'feature': x_features.cpu(), 'label': y.cpu()}
        test_df = test_df.append(new_row, ignore_index=True)
        i += 1
        if i == 5:
            break

# Saving the dataframes with extracted features
# save_folder = "/usr/itetnas04/data-scratch-01/ddordevic/data/cluster_scripts/vit_copy/extracted_features"
save_folder = "C:/Users/danil/Desktop/Master thesis/Code/msc-thesis/extracted_features"
training_df.to_pickle(save_folder + "/training_mnist.pkl")
test_df.to_pickle(save_folder + "/test_mnist.pkl")

print('Sucessfully saved the dataframes containing extracted features in pickle files.')

  0%|          | 0/60000 [00:00<?, ?it/s]

Using device:  cuda (NVIDIA GeForce MX450)


Sucessfully saved the dataframes containing extracted features in pickle files.


In [18]:
y[0]

tensor(4, device='cuda:0')

In [11]:
training_df

,image,feature,label
0,"[[[tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.,...","[[tensor(-19.0329), tensor(-1.0658), tensor(1....",[tensor(2)]
1,"[[[tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.,...","[[tensor(-10.0276), tensor(-17.7669), tensor(1...",[tensor(1)]
2,"[[[tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.,...","[[tensor(3.0456), tensor(-3.0474), tensor(-6.6...",[tensor(3)]
3,"[[[tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.,...","[[tensor(-18.3735), tensor(-4.6717), tensor(1....",[tensor(2)]
4,"[[[tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.,...","[[tensor(-4.8202), tensor(9.5271), tensor(2.45...",[tensor(4)]


In [6]:
new_row = {'image': 1, 'feature': 2, 'label': 3}
training_df.append(new_row, ignore_index=True)

,image,feature,label
0,1,2,3
